In [1]:
import os
import libtts

import numpy as np

import matplotlib.pyplot as plt

## ground detection

In [ ]:
# set input file
infile = f"tree_228.pts"

In [ ]:
# set output files
out_gd_file = f"tree_228_gd_xyzh.pts"
out_veg_file = f"tree_228_veg_xyzh.pts"
out_gd_file, out_veg_file = libtts.run_ground_detection(infile = infile, out_gd_file = out_gd_file, out_veg_file = out_veg_file, 
                                                        grid_size=0.1, height_threshold = 0.5)


In [ ]:
# plot gd points
gd_points = np.loadtxt(out_gd_file)
plt.figure(figsize=(7, 7))
plt.scatter(gd_points[:, 0], gd_points[:, 1], c=gd_points[:, 3], s=1, cmap='viridis')
plt.colorbar(label='Height (m)')
plt.title('Ground Points')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.axis('equal')
#plt.savefig('ground_points.png')
plt.show()
plt.close()

## tree detection and extraction

In [ ]:
tls_veg_file = "close_stems_3.pts"

In [ ]:
tree_locfile = libtts.run_tree_detection(infile = tls_veg_file, outfile= "tree_locations.pts", 
                                         height_min=0.5, height_max=1.0, max_dist=1.0,
                                         eps=0.2, min_samples=5)

print(tree_locfile)

In [ ]:
# plot tree locations
tree_locs = np.loadtxt(tree_locfile)
plt.figure(figsize=(7, 7))
plt.scatter(tree_locs[:, 0], tree_locs[:, 1], c=tree_locs[:, 3], s=10, label='Tree Locations')
plt.title('Tree Locations')
plt.xlabel('X (m)')
plt.ylabel('Y (m)')
plt.axis('equal')
plt.legend()
#plt.savefig('tree_locations.png')
plt.show()
plt.close()

## tree extraction

In [ ]:
### generate alpha shape
th_alpha_sq = 0.01
as_file = libtts.generate_alpha_shape(tls_veg_file, th_alpha_sq)
print("Alpha shape file:", as_file)

In [ ]:
### tree segmentation
as_file = f"{tls_veg_file[:-4]}_a0.010.off"
treeloc_file = f"{tls_veg_file[:-4]}_treeloc_xyzl.pts"
print(f"{as_file=}, {treeloc_file=}")

segfile = libtts.tls_extract_single_trees(as_file, treeloc_file, th_p2trunk_distance=0.2, th_search_radius=0.25)

print(segfile) 

In [ ]:
### label remaining points
tls_veg_file = "close_stems_3.pts"
segfile = f"{tls_veg_file[:-4]}_a0.010_lbl.pts"

all_pts = np.loadtxt(tls_veg_file)
initial_labels = np.loadtxt(segfile)
all_lbl_file = f"{tls_veg_file[:-4]}_lbl_all.pts"
all_pts_lbls = libtts.label_points_region_growing(all_pts, initial_labels, search_radius=0.5, out_file=all_lbl_file)

## downsampling

In [ ]:
infile = "t109_roi.pts"


In [ ]:
dsfile = libtts.run_downsampling(infile, input_data = "pts", th_alpha_sq=0.01, th_avg_dis=0.1)

print(dsfile)

vertices #:1454340
tetra #: 8467783
triangles #: 10918
edges #: 1157
Successfully wrote mesh to:
t109_roi_a0.010.off
Infile infos:
path: "/home/alex/Projects/libTTS_public/some_examples/t109_roi_a0.010.off"
file name: t109_roi_a0.010
extension: .off

Reading input
Entrato
off header: 1454340, 8479858, 0
scalar function id: 3
Complex Max Dim 3
Dim ID
1 0
2 1
3 2
Complex vertices #: 1454340
 points extent: x y min: -105.204, -59.669
Complex top simplices #: 8479858
Reading time 51.8166 s
Computing fields
Simplicial complex dimension 3
** Number of fields to create scalar function value: 1. **
Note - file ../data/table2D.txt cannot be found
xx: I will use embed values
Note - file ../data/table3D.txt cannot be found
xx: I will use embed values
Filtration time 0.767997 s

Compute Forman Gradient
Tops computed 14.941 s
Forman gradient computed 54.7485 s
The Forman gradient has critical cells: 
Dim: 0  #: 2476
Dim: 1  #: 10957
Dim: 2  #: 6745

Initialized 2476 clusters from critical minima.
P

In [4]:
as_file = "t109_roi_a0.010.off"
dsfile = libtts.run_downsampling(as_file, input_data = "mesh", th_alpha_sq=0.01, th_avg_dis=0.1)


Infile infos:
path: "/home/alex/Projects/libTTS_public/some_examples/t109_roi_a0.010.off"
file name: t109_roi_a0.010
extension: .off

Reading input
Entrato
Oversegmentation results saved to /home/alex/Projects/libTTS_public/some_examples/t109_roi_a0.010_lbl.pts
off header: 1454340, 8479858, 0
scalar function id: 3
Complex Max Dim 3
Dim ID
1 0
2 1
3 2
Complex vertices #: 1454340
 points extent: x y min: -105.204, -59.669
Complex top simplices #: 8479858
Reading time 51.3192 s
Computing fields
Simplicial complex dimension 3
** Number of fields to create scalar function value: 1. **
Note - file ../data/table2D.txt cannot be found
xx: I will use embed values
Note - file ../data/table3D.txt cannot be found
xx: I will use embed values
Filtration time 0.836628 s

Compute Forman Gradient
Tops computed 14.7961 s
Forman gradient computed 53.1689 s
The Forman gradient has critical cells: 
Dim: 0  #: 2476
Dim: 1  #: 10957
Dim: 2  #: 6745

Initialized 2476 clusters from critical minima.
Point label

In [5]:
overseg_file = "t109_roi_a0.010_lbl.pts"
dsfile = libtts.run_downsampling(overseg_file, input_data = "overseg", th_alpha_sq=0.01, th_avg_dis=0.1)

pts #: 1454340
labels #: 2476
selected_pts #: 1444398


## aux functions

In [ ]:
from plyfile import PlyData
import numpy as np

In [ ]:
def show_ply_point_summary(filepath):
    """
    Reads a PLY file and prints a summary of its point data.

    Args:
        filepath (str): The path to the PLY file.
    """
    try:
        plydata = PlyData.read(filepath)
        
        # show element names and properties
        print(f"Reading PLY file: {filepath}")
        print("Elements in the PLY file:")
        for element in plydata.elements:
            print(f"  - {element.name}: {len(element.data)} points")
            print(f"    Properties:")
            for prop in element.data.dtype.names:
                print(f"      - {prop}: {element.data.dtype[prop]}")
        print("\nPoint data summary:")

        element_names = [element.name for element in plydata.elements]
        if 'vertex' in element_names:
            vertex_data = plydata['vertex'].data
            print(f"Number of points: {len(vertex_data)}")

            # # Print information about the properties (e.g., x, y, z, color)
            # print("Point properties and their data types:")
            # for prop_name in vertex_data.dtype.names:
            #     print(f"  - {prop_name}: {vertex_data.dtype[prop_name]}")

            # Print min/max values for specific properties 
            # if # of pts is relatively small, otherwise it may take too long
            if len(vertex_data) < 1000000 and \
               'x' in vertex_data.dtype.names and \
               'y' in vertex_data.dtype.names and \
               'z' in vertex_data.dtype.names:
                print("\nCoordinate ranges:")
                print(f"  X-range: [{vertex_data['x'].min():.4f}, {vertex_data['x'].max():.4f}]")
                print(f"  Y-range: [{vertex_data['y'].min():.4f}, {vertex_data['y'].max():.4f}]")
                print(f"  Z-range: [{vertex_data['z'].min():.4f}, {vertex_data['z'].max():.4f}]")
        else:
            print(f"No 'vertex' element found in '{filepath}'.")
            print("Available elements:", [el.name for el in plydata.elements])

    except FileNotFoundError:
        print(f"Error: File not found at '{filepath}'")
    except Exception as e:
        print(f"An error occurred: {e}")

In [ ]:
show_ply_point_summary("/home/alex/Projects/libTTS_public/cpp/cmake-build-debug/close_stems_3_a0.01.ply")

In [ ]:
show_ply_point_summary("/home/alex/Projects/libTTS_public/cpp/cmake-build-debug/close_stems_3_a0.01_lbl.ply")

In [ ]:
show_ply_point_summary("/run/media/alex/HDD_Data3/GoogleDrive/ResearchDevelopment/projects/john_postdoc_2024/test_data/aoi_vg_xyzh_a0.100.ply")